In [1]:
import io
import os
import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics

import tensorflow as tf
from tensorflow.keras import Input, Model, layers, losses, optimizers, callbacks
import tensorflow_decision_forests as tfdf

default_max_columns = None
default_max_rows = 10
pd.set_option('display.max_columns', default_max_columns)
pd.set_option('display.max_rows', default_max_rows)

df_name = "WUSTL-IIOT"

2025-02-02 15:45:21.636097: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-02 15:45:21.715500: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-02 15:45:21.789362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-02 15:45:21.859371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-02 15:45:21.879634: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-02 15:45:21.992642: I tensorflow/core/platform/cpu_feature_gu

In [2]:
#Utility functions
protocol_dict = {"0":"HOPOPT",
                 "1":"ICMP",
                 "2":"IGMP",
                 "6":"TCP",
                 "17":"UDP",
                 "58":"IPv6-ICMP",
                 "2054":"ARP",
                 "35020":"LLDP"
                }

In [3]:
#Load Data
file_path = "" #File paths deleted for privacy reasons

df_raw = pd.read_csv(file_path, dtype=str)

In [4]:
df_raw.shape

(1194464, 49)

In [5]:
df_raw.head(5)

,StartTime,LastTime,SrcAddr,DstAddr,Mean,Sport,Dport,SrcPkts,DstPkts,TotPkts,DstBytes,SrcBytes,TotBytes,SrcLoad,DstLoad,Load,SrcRate,DstRate,Rate,SrcLoss,DstLoss,Loss,pLoss,SrcJitter,DstJitter,SIntPkt,DIntPkt,Proto,Dur,TcpRtt,IdleTime,Sum,Min,Max,sDSb,sTtl,dTtl,sIpId,dIpId,SAppBytes,DAppBytes,TotAppByte,SynAck,RunTime,sTos,SrcJitAct,DstJitAct,Traffic,Target
0,2019-08-19 12:23:28,2019-08-19 12:23:28,192.168.0.20,192.168.0.2,0,59034,502,10,8,18,508,644,1152,87486.09375,67122.953125,154609.046875,169.692856,131.983337,320.530945,2,2,4,18.181818,527.431726,11.523097,5.893,7.406429,6,0.053037,0.001266,1548786176,0.053037,0.053037,0.053037,0,128,64,53331,64402,24,20,44,0.001176,0.053037,0,0,0,normal,0
1,2019-08-19 15:13:24,2019-08-19 15:13:24,192.168.0.20,192.168.0.2,0,55841,502,10,8,18,508,644,1152,88077.296875,67576.546875,155653.84375,170.839584,132.875229,322.696991,2,2,4,18.181818,17.234379,13.246678,7.525857,7.338714,6,0.052681,0.00131,1548882816,0.052681,0.052681,0.052681,0,128,64,37167,31590,24,20,44,0.001308,0.052681,0,0,0,normal,0
2,2019-08-19 13:41:31,2019-08-19 13:41:31,192.168.0.20,192.168.0.2,0,63774,502,10,8,18,508,644,1152,89587.390625,68735.15625,158322.546875,173.768646,135.153397,328.229675,2,2,4,18.181818,522.98724,12.307223,5.754778,7.299143,6,0.051793,0.000766,1548877312,0.051793,0.051793,0.051793,0,128,64,58712,22717,24,20,44,0.00069,0.051793,0,0,0,normal,0
3,2019-08-19 12:43:19,2019-08-19 12:43:20,209.240.235.92,192.168.0.2,0,61771,80,4,0,4,0,248,248,1672.746582,0,1672.746582,3.372473,0,3.372473,3,0,3,42.857143,419.338813,0,296.518344,0,6,0.889555,0,1548787456,0.889555,0.889555,0.889555,0,140,0,21629,0,0,0,0,0,0.889555,0,419.338813,0,DoS,1
4,2019-08-19 14:49:44,2019-08-19 14:49:48,192.168.0.20,192.168.0.1,3,0,0,14,0,14,0,868,868,1842.256714,0,1842.256714,3.714227,0,3.714227,0,0,0,0,525.146562,0,321.429844,0,2054,3.500055,0,1548881408,3.500055,3.500055,3.500055,0,0,0,0,0,476,0,476,0,3.500055,0,525.146562,0,normal,0


In [6]:
df_raw.describe()

,StartTime,LastTime,SrcAddr,DstAddr,Mean,Sport,Dport,SrcPkts,DstPkts,TotPkts,DstBytes,SrcBytes,TotBytes,SrcLoad,DstLoad,Load,SrcRate,DstRate,Rate,SrcLoss,DstLoss,Loss,pLoss,SrcJitter,DstJitter,SIntPkt,DIntPkt,Proto,Dur,TcpRtt,IdleTime,Sum,Min,Max,sDSb,sTtl,dTtl,sIpId,dIpId,SAppBytes,DAppBytes,TotAppByte,SynAck,RunTime,sTos,SrcJitAct,DstJitAct,Traffic,Target
count,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464,1194464
unique,25267,25276,14,132,6,51057,7781,567,321,637,446,1592,1742,84158,65827,107725,82185,46482,89076,68,135,152,274,1103303,941127,99642,53631,8,81931,4752,200,81522,81522,81522,5,255,3,44717,65536,468,277,933,4021,81522,5,57226,4293,5,2
top,2019-08-19 12:32:29,2019-08-19 12:29:43,192.168.0.20,192.168.0.2,0,0,502,10,8,18,508,644,1152,4.96e+08,0,4.96e+08,1e+06,0,1e+06,2,2,4,18.181818,0,0,0.001,0,6,1e-06,0,1548786432,1e-06,1e-06,1e-06,0,128,64,21629,0,24,20,44,0,1e-06,0,0,0,normal,0
freq,312,351,1090574,1155810,1149300,12281,1103533,923271,914111,912505,880644,921747,878797,28963,108051,28985,32175,108048,32425,1118620,1077678,1089742,911834,58780,112091,26138,107982,1153955,32426,115628,12387,32426,32426,32426,1194076,1091358,1083642,46637,108752,1076920,1016120,1015888,115629,32426,1194076,1136748,1190159,1107448,1107448


In [7]:
#Clean
df_cleaned = df_raw

#Renaming
df_cleaned = df_cleaned.rename(columns={
    "SrcAddr":"IPV4_SRC_ADDR",
    "DstAddr":"IPV4_DST_ADDR",
    "Sport":"L4_SRC_PORT",
    "Dport":"L4_DST_PORT",
    "Dur":"FLOW_DURATION_MILLISECONDS",
    "Proto":"PROTOCOL",
    "SrcBytes":"OUT_BYTES",
    "DstBytes":"IN_BYTES",
    "SrcPkts":"OUT_PKTS",
    "DstPkts":"IN_PKTS", 
    "Target":"Label"
})

print("invalid ip adresses count:")
x = df_cleaned.loc[df_cleaned["IPV4_SRC_ADDR"] == "0"].shape[0]
print(x)
print(x/df_raw.shape[0])
print("IPV6 adresses count:")
x = df_cleaned.loc[df_cleaned["IPV4_SRC_ADDR"].str.contains(":",  na = False)].shape[0]
print(x)
print(x/df_raw.shape[0])


df_cleaned = df_cleaned.drop(df_cleaned.loc[df_cleaned["IPV4_SRC_ADDR"] == "0"].index)
df_cleaned = df_cleaned.drop(df_cleaned.loc[df_cleaned["IPV4_SRC_ADDR"].str.contains(":",  na = False)].index) # IPV 6 adresses deleted

invalid ip adresses count:
423
0.0003541337369732365
IPV6 adresses count:
3220
0.002695769818094141


In [8]:
df_cleaned.head(5)

,StartTime,LastTime,IPV4_SRC_ADDR,IPV4_DST_ADDR,Mean,L4_SRC_PORT,L4_DST_PORT,OUT_PKTS,IN_PKTS,TotPkts,IN_BYTES,OUT_BYTES,TotBytes,SrcLoad,DstLoad,Load,SrcRate,DstRate,Rate,SrcLoss,DstLoss,Loss,pLoss,SrcJitter,DstJitter,SIntPkt,DIntPkt,PROTOCOL,FLOW_DURATION_MILLISECONDS,TcpRtt,IdleTime,Sum,Min,Max,sDSb,sTtl,dTtl,sIpId,dIpId,SAppBytes,DAppBytes,TotAppByte,SynAck,RunTime,sTos,SrcJitAct,DstJitAct,Traffic,Label
0,2019-08-19 12:23:28,2019-08-19 12:23:28,192.168.0.20,192.168.0.2,0,59034,502,10,8,18,508,644,1152,87486.09375,67122.953125,154609.046875,169.692856,131.983337,320.530945,2,2,4,18.181818,527.431726,11.523097,5.893,7.406429,6,0.053037,0.001266,1548786176,0.053037,0.053037,0.053037,0,128,64,53331,64402,24,20,44,0.001176,0.053037,0,0,0,normal,0
1,2019-08-19 15:13:24,2019-08-19 15:13:24,192.168.0.20,192.168.0.2,0,55841,502,10,8,18,508,644,1152,88077.296875,67576.546875,155653.84375,170.839584,132.875229,322.696991,2,2,4,18.181818,17.234379,13.246678,7.525857,7.338714,6,0.052681,0.00131,1548882816,0.052681,0.052681,0.052681,0,128,64,37167,31590,24,20,44,0.001308,0.052681,0,0,0,normal,0
2,2019-08-19 13:41:31,2019-08-19 13:41:31,192.168.0.20,192.168.0.2,0,63774,502,10,8,18,508,644,1152,89587.390625,68735.15625,158322.546875,173.768646,135.153397,328.229675,2,2,4,18.181818,522.98724,12.307223,5.754778,7.299143,6,0.051793,0.000766,1548877312,0.051793,0.051793,0.051793,0,128,64,58712,22717,24,20,44,0.00069,0.051793,0,0,0,normal,0
3,2019-08-19 12:43:19,2019-08-19 12:43:20,209.240.235.92,192.168.0.2,0,61771,80,4,0,4,0,248,248,1672.746582,0,1672.746582,3.372473,0,3.372473,3,0,3,42.857143,419.338813,0,296.518344,0,6,0.889555,0,1548787456,0.889555,0.889555,0.889555,0,140,0,21629,0,0,0,0,0,0.889555,0,419.338813,0,DoS,1
4,2019-08-19 14:49:44,2019-08-19 14:49:48,192.168.0.20,192.168.0.1,3,0,0,14,0,14,0,868,868,1842.256714,0,1842.256714,3.714227,0,3.714227,0,0,0,0,525.146562,0,321.429844,0,2054,3.500055,0,1548881408,3.500055,3.500055,3.500055,0,0,0,0,0,476,0,476,0,3.500055,0,525.146562,0,normal,0


In [9]:
#Preprocessing
df_processed = df_cleaned

#Protocol Decoding
df_processed.loc[:,"PROTOCOL"] = df_processed["PROTOCOL"].apply(lambda x: protocol_dict[str(x)])

In [10]:
df_processed.head(5)

,StartTime,LastTime,IPV4_SRC_ADDR,IPV4_DST_ADDR,Mean,L4_SRC_PORT,L4_DST_PORT,OUT_PKTS,IN_PKTS,TotPkts,IN_BYTES,OUT_BYTES,TotBytes,SrcLoad,DstLoad,Load,SrcRate,DstRate,Rate,SrcLoss,DstLoss,Loss,pLoss,SrcJitter,DstJitter,SIntPkt,DIntPkt,PROTOCOL,FLOW_DURATION_MILLISECONDS,TcpRtt,IdleTime,Sum,Min,Max,sDSb,sTtl,dTtl,sIpId,dIpId,SAppBytes,DAppBytes,TotAppByte,SynAck,RunTime,sTos,SrcJitAct,DstJitAct,Traffic,Label
0,2019-08-19 12:23:28,2019-08-19 12:23:28,192.168.0.20,192.168.0.2,0,59034,502,10,8,18,508,644,1152,87486.09375,67122.953125,154609.046875,169.692856,131.983337,320.530945,2,2,4,18.181818,527.431726,11.523097,5.893,7.406429,TCP,0.053037,0.001266,1548786176,0.053037,0.053037,0.053037,0,128,64,53331,64402,24,20,44,0.001176,0.053037,0,0,0,normal,0
1,2019-08-19 15:13:24,2019-08-19 15:13:24,192.168.0.20,192.168.0.2,0,55841,502,10,8,18,508,644,1152,88077.296875,67576.546875,155653.84375,170.839584,132.875229,322.696991,2,2,4,18.181818,17.234379,13.246678,7.525857,7.338714,TCP,0.052681,0.00131,1548882816,0.052681,0.052681,0.052681,0,128,64,37167,31590,24,20,44,0.001308,0.052681,0,0,0,normal,0
2,2019-08-19 13:41:31,2019-08-19 13:41:31,192.168.0.20,192.168.0.2,0,63774,502,10,8,18,508,644,1152,89587.390625,68735.15625,158322.546875,173.768646,135.153397,328.229675,2,2,4,18.181818,522.98724,12.307223,5.754778,7.299143,TCP,0.051793,0.000766,1548877312,0.051793,0.051793,0.051793,0,128,64,58712,22717,24,20,44,0.00069,0.051793,0,0,0,normal,0
3,2019-08-19 12:43:19,2019-08-19 12:43:20,209.240.235.92,192.168.0.2,0,61771,80,4,0,4,0,248,248,1672.746582,0,1672.746582,3.372473,0,3.372473,3,0,3,42.857143,419.338813,0,296.518344,0,TCP,0.889555,0,1548787456,0.889555,0.889555,0.889555,0,140,0,21629,0,0,0,0,0,0.889555,0,419.338813,0,DoS,1
4,2019-08-19 14:49:44,2019-08-19 14:49:48,192.168.0.20,192.168.0.1,3,0,0,14,0,14,0,868,868,1842.256714,0,1842.256714,3.714227,0,3.714227,0,0,0,0,525.146562,0,321.429844,0,ARP,3.500055,0,1548881408,3.500055,3.500055,3.500055,0,0,0,0,0,476,0,476,0,3.500055,0,525.146562,0,normal,0


In [11]:
#Export
df_processed.to_csv(df_name+"_pre_aligned.csv",index=False)